# Stress profile evaluation

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import \
    DICGrid, DICInpUnstructuredPoints, DICStateFields, \
    DICAlignedGrid, DICGrid
from bmcs_shear.dic_crack.dic_crack_list2 import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

In [ ]:
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B6_TV1', t=1,
                                      n_T_max=40, T_stepping='delta_T')
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, d_x=5, d_y=5)

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, R=6, omega_threshold=0.15, verbose_eval=True)
dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                      epsilon_f=0.0019, eps_max=0.01);
dsf = DICStateFields(dic_grid=dic_grid, R=8, omega_threshold=0.15, verbose_eval=True, tmodel='scalar_damage')
dsf.tmodel_.trait_set(E=1600, nu=0.18, omega_fn='exp-slope', strain_norm='Rankine', eps_max=0.01)
dsf.tmodel_.omega_fn_.trait_set(kappa_0=0.002, kappa_f=0.0028);
dcl = DICCrackList(dsf=dsf, 
                   t_detect=0.95,
                   delta_alpha_min=-0.5, # -np.pi/6, 
                   delta_alpha_max=np.pi/3, delta_s = 25, x_boundary=30)

Interactive crack detection procedure

 - define the length of the crack segment for the processing
 - define the angle corridor to search the next localized crack
 - define the load level for primary crack detection
 

In [ ]:
dcl.interact()

# Introspect the results

In [ ]:
dcl.t_detect=0.8
dcl.primary_cracks

In [ ]:
dcl.dsf.bd.matrix_.trait_set(
    f_c = 38, f_t = 2.97, E_c = 26800
)
cbda = dcl.dsf.bd.cross_section_layout.items['crack bridge/dowel action']
cbda.trait_set(E_s=199000, tau=10, sig_y=579)

In [ ]:
cbda.interact()

In [ ]:
cbda.get_Pw_pull(0.5)

In [ ]:
c = 0
sp = dcl.cracks[c].sp

In [ ]:
sp.interact()

In [ ]:
import bmcs_utils.api as bu
%matplotlib widget
fig, ax_F = plt.subplots(1,1)
ax_sig = ax_F.twiny()
sp.plot_sig_t_crc_La(ax_sig)
sp.plot_sig_t_unc_Lab(ax_sig)
sp.plot_F_t_a(ax_F)
bu.mpl_align_xaxis(ax_sig, ax_F)

In [ ]:
self = sp
X_mid_unc_t_a = np.average(self.X_unc_t_La, axis=0)
# horizontal distance between dowel action and crack tip.
x_La = self.X_crc_t_La
x_rot, y_rot = X_mid_unc_t_a
delta_x_crc_La = x_rot - x_La[:, 0]# - x_rot
delta_y_crc_La = y_rot - x_La[:, 1]
B = sp.dic_grid.sz_bd.B
M_L_0 = np.trapz(delta_y_crc_La * self.sig_crc_t_La[:, 0], x_La[:,1]) * B
M_L_1 = np.trapz(delta_x_crc_La * self.sig_crc_t_La[:, 1], x_La[:,0]) * B
M = np.sum(M_L_0, axis=0) + np.sum(M_L_1, axis=0)
M

In [ ]:
delta_y_N = y_rot - self.z_N
M_y = np.einsum('i,i', delta_y_N, self.F_t_Na[:, 0])
# assuming that the horizontal position of the crack bridge
# is almost equal to the initial position of the crack x_00
# x_00 = np.ones_like(self.z_N) * self.sz_cp.x_00
x_00 = self.dic_crack.C_cubic_spline(self.z_N)
delta_x_N = x_00 - x_rot
M_da = np.einsum('i,i', delta_x_N, self.F_t_Na[:, 1])
M_y, M_da, M

In [ ]:
sp.M_mid_unc_t_a

In [ ]:
sp.M_ext_kN_t * 1000

In [ ]:
M_cb_t_ = []
M_da_t_ = []
M_ag_t_ = []
M_ext_t_ = []
print('crack ', end='')
c = 2
crack = dcl.cracks[c]
sp = crack.sp
t_range = np.linspace(0,1,10)
for t in t_range:
    dcl.dsf.dic_grid.t = t
    M_cb, M_da, M_ag = sp.M_mid_unc_t_a
    M_cb_t_.append(M_cb/1000)
    M_da_t_.append(M_da/1000)
    M_ag_t_.append(M_ag/1000)
    M_ext_t_.append(self.M_ext_kN_t)
M_cb_t = np.array(M_cb_t_)
M_da_t = np.array(M_da_t_)
M_ag_t = np.array(M_ag_t_)
M_ext_t = np.array(M_ext_t_)

In [ ]:
t_range, M_cb_t

In [ ]:
%matplotlib widget
fig, (ax, ax_V) = plt.subplots(1,2, figsize=(8,3))
colors = ['red', 'blue', 'green', 'magenta', 'brown', 'gray', 'yellow']
ax.plot(t_range, M_cb_t, color='red')
#ax.fill_between(t_range, M_cb_t, 0, 'red', alpha=0.2) 
ax.plot(t_range, M_da_t + M_cb_t, color='red')
#ax.fill_between(t_range, M_da_t, M_cb_t, 'red', alpha=0.2) 
ax.plot(t_range, M_ag_t + M_da_t + M_cb_t, color='red')
#ax.fill_between(t_range, M_ag_t, M_da_t, 'red', alpha=0.2) 
ax.plot(t_range, M_ext_t, color='black')
#ax.legend();

# Check to see if the global moment equilibrium is related to the correct point

In [ ]:
dcl.cracks

In [ ]:
V_crc_Ct = []
V_unc_Ct = []
V_ext_Ct = []
print('crack ', end='')
for c, crack in enumerate(dcl.cracks):
    print('{}, '.format(c), end='')
    sp = crack.sp
    V_crc_t_ = []
    V_unc_t_ = []
    V_ext_t_ = []
    t_range = np.linspace(0,1,10)
    for t in t_range:
        dcl.dsf.dic_grid.t = t
        V_crc_t_.append(sp.V_crc_y)
        V_unc_t_.append(sp.V_unc_y)
        V_ext_t_.append(sp.V_ext_kN_t)
    V_ext_Ct.append(np.array(V_ext_t_))
    V_crc_Ct.append(np.array(V_crc_t_)/1000)
    V_unc_Ct.append(np.array(V_unc_t_)/1000)
    

In [ ]:
sp.dic_grid.F_T_t

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
colors = ['red', 'blue', 'green', 'magenta', 'brown', 'gray', 'yellow']
for c, (color, V_crc_t, V_ext_t) in enumerate(zip(colors, V_crc_Ct, V_ext_Ct)):
    ax.plot(t_range, V_crc_t[:,0], color=color, label='crack {}'.format(c))
    ax.fill_between(t_range, V_crc_t[:,0], 0, color=color, alpha=0.1) 
    ax.plot(t_range, V_ext_t, linestyle='dotted', color='black')
ax.plot(t_range, V_ext_t)
ax.legend();

In [ ]:
sp.V_crc_y

# Shear stress profile evaluated from tensile profile

## Access the stress or strain field

In [ ]:
t = 1
sig_d_x = 10
c = dcl.cracks[0]

c.dic_grid.t = t
sig2_d_x = sig_d_x / 2
x_K, y_K = c.X_unc_t_Ka.T
t_K = np.ones_like(x_K) * t

tX_right_K = np.array([t_K, x_K + sig2_d_x, y_K], dtype=np.float_).T
tX_left_K = np.array([t_K, x_K - sig2_d_x, y_K], dtype=np.float_).T
y_K

In [ ]:
E_, nu_ = 28000, 0.2

In [ ]:
eps_left_Kab, eps_right_Kab = dcl.dsf.f_eps_fe_txy(tX_left_K), dcl.dsf.f_eps_fe_txy(tX_right_K)

In [ ]:
_1_nu_ = 1 / (1 - nu_)
E_ab = E_ / (1 + nu_) * np.array([[ _1_nu_, _1_nu_, 0],
                                  [ _1_nu_, _1_nu_, 0],
                                  [ 0, 0, 0.5]])

In [ ]:
eps_left_Kd = np.hstack([eps_left_Kab[:, [0, 1], [0, 1]], 2 * eps_left_Kab[:, 0, 1, np.newaxis]])
eps_right_Kd = np.hstack([eps_right_Kab[:, [0, 1], [0, 1]], 2 * eps_right_Kab[:, 0, 1, np.newaxis]])

In [ ]:
sig_left_Kd = np.einsum('cd, ...d->...c', E_ab, eps_left_Kd)
sig_right_Kd = np.einsum('cd, ...d->...c', E_ab, eps_right_Kd)

In [ ]:
delta_sig_Kd = sig_right_Kd - sig_left_Kd

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
fig.canvas.header_visible = False
eps_K_ = sig_right_Kd[:, 0]
ax.plot(eps_K_, y_K)
sp.plot_sig_t_unc_Lab(ax)
ax.plot(np.zeros_like(eps_K_), y_K)